# Test01: test code for operations (+,*)

# Preparation 

## Preparation of google drive (if required)

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更して下さい。

In [3]:
%cd "ctorch220301/MiniCTorch_Prototype/notebook"

import sys
sys.path.append( "../")

/content/drive/My Drive/Colab Notebooks/ctorch220301/MiniCTorch_Prototype/notebook


## Installation of MiniCTorch

In [4]:
! pip install lark-parser

     |████████████████████████████████| 103 kB 27.2 MB/s 


In [ ]:
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

  Cloning https://github.com/kojima-r/MiniCTorch_Prototype.git to /tmp/pip-req-build-nmxii7ck
  Running command git clone -q https://github.com/kojima-r/MiniCTorch_Prototype.git /tmp/pip-req-build-nmxii7ck


## ニューラルネットの定義

In [5]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch

In [6]:
class Net(nn.Module):
   def __init__(self):
      super(Net, self).__init__()
      self.t = torch.tensor(np.array([[1.0, 2], [3, 4]]))
           
   def forward(self, x):
      f1 = 10 * x * x * self.t
      f2 = 5 * x
      y = f1 + f2
      return y

## プロジェクト名、作業フォルダ、JSONファイル名の設定

In [7]:
project = "test01"
folder = "./test"
json_path = folder + '/' + project +'.json'

In [8]:
! mkdir -p test

## 計算グラフをJSONファイルに出力する

In [9]:
def generate_json( json_path, input_x ):
   model = Net()
   model.eval()
   with torch.no_grad():
      minictorch.trace( model, input_x, json_path )
   return model

In [10]:
x = torch.tensor(np.array([[1.0, 2], [3, 4]]), requires_grad=True)
print(x)
model = generate_json( json_path, x )

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64, requires_grad=True)


JSONファイルをC++コードに変換する

In [11]:
minictorch.convert_all("test01", "./test", model, json_path, x, {"inp_data":x}, code="test" )

[JSON] ./test/test01.json
param: xin  - str loop  0  /  0
{'name': 'Net/7', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [2, 2], 'constant_value': [1.0, 2.0, 3.0, 4.0], 'out': [5], 'sorted_id': 4}
param: Constant1  - str loop  0  /  0
[PARAM] ./test/test01_param.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'output_id': 0, 'shape': [2, 2], 'out': [3, 2, 7], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [], 'constant_value': 10.0, 'out': [2], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'aten::mul', 'in': [0, 1], 'output_id': 0, 'shape': [2, 2], 'out': [3], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'aten::mul', 'in': [2, 0], 'output_id': 0, 'shape': [2, 2], 'out': [5], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'prim::Constant', 'in': [], 'output_id': 0, 'shape': [2, 2], 'constant_value': [1.0, 2.0, 3.0, 4.0], 'out': [5], 'sorted_id': 4}
{'name': 'Net/f1', 'op': 'aten::mul', 'in': [3, 4], 'output_id': 0, 'shape': [2, 2], 'out': [9]

pythonコードを実行する

In [12]:
# check code
model.train()
y = model( x )
print(y)
y.sum().backward()
print(x.grad)

tensor([[ 15.,  90.],
        [285., 660.]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[ 25.,  85.],
        [185., 325.]], dtype=torch.float64)


## 環境構築
(注意) インストール済みの場合は各自の環境に合わせて変更して下さい。

In [13]:
! git clone https://github.com/xtensor-stack/xtensor.git
! git clone https://github.com/xtensor-stack/xtensor-blas.git
! git clone https://github.com/xtensor-stack/xtl.git
! git clone https://github.com/nlohmann/json.git


Cloning into 'xtensor'...
remote: Enumerating objects: 20434, done.
remote: Counting objects: 100% (3462/3462), done.
remote: Compressing objects: 100% (517/517), done.
remote: Total 20434 (delta 3010), reused 3201 (delta 2928), pack-reused 16972
Receiving objects: 100% (20434/20434), 10.10 MiB | 4.42 MiB/s, done.
Resolving deltas: 100% (14580/14580), done.
Checking out files: 100% (385/385), done.
Cloning into 'xtensor-blas'...
remote: Enumerating objects: 3857, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 3857 (delta 83), reused 96 (delta 41), pack-reused 3688
Receiving objects: 100% (3857/3857), 4.13 MiB | 9.05 MiB/s, done.
Resolving deltas: 100% (3155/3155), done.
Checking out files: 100% (1435/1435), done.
Cloning into 'xtl'...
remote: Enumerating objects: 2399, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 2399 (delta 125), reused 137 

In [ ]:
! git clone https://github.com/kojima-r/MiniCTorch_Prototype.git

Cloning into 'MiniCTorch_Prototype'...
remote: Enumerating objects: 728, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (359/359), done.
remote: Total 728 (delta 499), reused 583 (delta 367), pack-reused 0
Receiving objects: 100% (728/728), 42.78 MiB | 10.71 MiB/s, done.
Resolving deltas: 100% (499/499), done.


## C++コードをコンパイル、実行

In [14]:
C_SRC = './test/test01.cpp ./test/test01_param.cpp'
C_OPT = '-I ../src -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include'
C_OUT = './test/test01'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./test/test01.cpp ./test/test01_param.cpp
OPT = -I ../src -I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./test/test01


In [15]:
! ./test/test01

### forward computation ...
{{  15.,   90.},
 { 285.,  660.}}
### backward computation ...
input_grad{{  25.,   85.},
 { 185.,  325.}}
